# Random Forest-modell för min voting class


## Importera nödvändiga paket.

In [ ]:
# Av eget intresse vill jag gärna veta hur lång tid olika saker tar.
import time
notebook_start = time.time()  
t0 = time.time()

# Skippa varningar
import warnings
warnings.filterwarnings('ignore')

# Boosta prestandan
from sklearnex import patch_sklearn
patch_sklearn()

# Paket för datahantering
import numpy as np
import pandas as pd
import os
import math

# Dataset och modeller
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split 

# Preprocessing/pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.kernel_approximation import Nystroem

# Dimensionsreducering
from sklearn.decomposition import PCA

# Feature-bearbetning
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

# Data augmentation och förbehandling - stabil version för SciPy
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter, map_coordinates

# Bildbehandling
from PIL import Image

# Modeller 
# from sklearn.svm import SVC, LinearSVC
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# För export av modellen/scalern för vidare användning i Streamlit-appen
import joblib
from joblib import Parallel, delayed

# Slutrapport
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Skona datorn
os.environ['OMP_NUM_THREADS'] = '8' 
os.environ['MKL_NUM_THREADS'] = '8'

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

## Läs in MNISt-datasetet och splitta det. 

*//Best practice: splitta ut testsetet direkt//*

In [ ]:
t0 = time.time()

# Läs in alla MNIST-data
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False, parser='auto')
X = mnist["data"]              
y = mnist["target"].astype(np.uint8)

# Splitta (80/20) med stratifiering för jämna klassfördelningar
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# Normalisering (en riktigt bra grej för både PCA och SVC!)
# Genom att dela med 255.0 blir alla värden mellan 0 och 1
X_train = X_train / 255.0
X_test = X_test / 255.0

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print("-" * 40)
print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

HoG ska boosta accuracy!

In [ ]:
class HogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def local_hog(img_flat):
            return hog(img_flat.reshape(28, 28), 
                       orientations=self.orientations, 
                       pixels_per_cell=self.pixels_per_cell, 
                       cells_per_block=self.cells_per_block,
                       visualize=False)
        
        return np.array([local_hog(x) for x in X])

Definitioner

In [ ]:
t0 = time.time()

def deskew(image):
    # Räta upp lutande siffror 
    img = image.reshape(28, 28)
    
    # Skapa koordinat-matriser (y för rader, x för kolumner)
    y, x = np.mgrid[:28, :28]
    
    # Hitta tyngdpunkten (Center of Mass) med ndimage
    mu = ndimage.center_of_mass(img)
    if np.isnan(mu).any(): # Om bilden är tom
        return img.flatten()
    
    # Beräkna centrala moments (mu11 = kovarians, mu02 = varians i y-led)
    mu11 = np.sum((x - mu[1]) * (y - mu[0]) * img)
    mu02 = np.sum((y - mu[0])**2 * img)
    
    # Om variansen är för liten lutar siffran inte eller är för tunn
    if abs(mu02) < 1e-2:
        return img.flatten()
    
    # Skew-faktorn (förskjutning av x per enhet y)
    skew = mu11 / mu02
    
    # Här rätas x upp genom subtrahering av skew * y
    # Matrisen blir [[1, 0], [skew, 1]] med SciPys omvända ordning.
    matrix = np.array([[1, 0], [skew, 1]])
    
    # Offset för att rotera/skeva kring bildens centrum (14, 14)
    center = np.array([14, 14])
    offset = center - np.dot(matrix, center)
    
    # Transformation
    img_deskewed = ndimage.affine_transform(img, matrix, offset=offset, order=1, mode='constant', cval=0)
    return img_deskewed.flatten()

def shift_image(image, dx, dy):
    return ndimage.shift(image.reshape(28, 28), [dy, dx], cval=0, mode="constant").flatten()

def rotate_image(image, angle):
    return ndimage.rotate(image.reshape(28, 28), angle, reshape=False, cval=0, mode="constant").flatten()

def zoom_image(image, factor):
    rescaled = ndimage.zoom(image.reshape(28, 28), factor)
    if factor > 1.0: # Zooma in (klipp)
        start = int((rescaled.shape[0] - 28) / 2)
        final = rescaled[start:start+28, start:start+28]
    else: # Zooma ut (padda)
        pad = int((28 - rescaled.shape[0]) / 2)
        final = np.pad(rescaled, ((pad, 28-rescaled.shape[0]-pad), (pad, 28-rescaled.shape[1]-pad)), mode='constant')
    return final.flatten()

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

# Läs in augmenterade data

In [ ]:
# --- NY CELL 6: LADDA FÄRDIGT DATA FRÅN C: ---
import joblib
import time

t0 = time.time()
load_path = "C:/mnist_data/mnist_augmented_boosted_elastic.joblib"

print(f"Laddar {load_path}...")

# Här hämtar vi allt SVC-modellen precis "lärde sig"
X_train_augmented, y_train_augmented = joblib.load(load_path)

print("-" * 40)
print(f"Antal rader laddade: {len(X_train_augmented)}")
print(f"Tid för inläsning: {time.time() - t0:.1f} sekunder")

Preparera testsetet så att jämförelsen blir rättvis.

In [ ]:
t0 = time.time()

# Här rätas testbilderna upp för att matcha modellen
X_test_deskewed = np.array([deskew(img) for img in X_test])

X_test_deskewed = X_test_deskewed.astype('float32')

cell_time = time.time() - t0
print("-" * 40)
print(f">>> Sökningen klar på {cell_time:.1f} sekunder.")

## Random Forest-pipeline

In [ ]:
t0 = time.time()

# Detta är vår "Expert-Pipeline" för Random Forest
# Vi kombinerar PCA och HOG för att ge träden de bästa geometriska ledtrådarna
final_pipe_rf = Pipeline([
    ('feature', FeatureUnion([
        ('pca', PCA(n_components=65)), # RF är duktig på att välja features, 100 räcker ofta
        ('hog', HogTransformer())
    ])),
    ('scaler', StandardScaler()), 
    ('rf', RandomForestClassifier(
        n_estimators=1000,      # Vi bygger en stor skog för max precision
        max_features='sqrt',   # Standard för klassificering
        criterion='gini',
        min_samples_leaf=1,
        bootstrap=True,
        oob_score=True,
        n_jobs=10,             # Skona datorn
        random_state=42,
        verbose=1              # Så vi ser att skogen växer fram
    ))
])

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Random Forest-Pipeline definierad! Tid: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

In [ ]:
# --- CELL 11: TRÄNING OCH UTVÄRDERING (RANDOM FOREST) ---
import gc 
import time

t0 = time.time()

# 1. Förbered data (Ingen kopiering behövs då den laddades som float32)
X_train_final = X_train_augmented 
y_train_final = y_train_augmented

# 2. RENSA MINNET (Bara en gång!)
# Vi tar bort referensen till listan för att ge RF-algoritmen maximalt RAM
if 'X_train_augmented' in locals():
    del X_train_augmented
gc.collect() 

# 3. Träna Random Forest-modellen
print(f"\n>>> STARTAR TRÄNING (Building Forest): {len(X_train_final)} rader.")
print(f">>> Använder n_components=65 (baserat på din sweep-analys).")

final_pipe_rf.fit(X_train_final, y_train_final)
print("\nSUCCESS! Skogen är färdigvuxen.")

# 4. Slutgiltig utvärdering
print("\n>>> STARTAR SCORE: Utvärderar testsetet...")
base_accuracy = final_pipe_rf.score(X_test_deskewed, y_test)

# Beräkna antal fel
y_pred_tmp = final_pipe_rf.predict(X_test_deskewed)
num_errors = np.sum(y_pred_tmp != y_test)

print("\n" + "="*40)
print(f"RESULTAT: RANDOM FOREST EXPERT")
print("-"*40)
print(f"Accuracy:      {base_accuracy:.5f}")
print(f"Antal fel:     {num_errors} av {len(y_test)}")
print("="*40)

cell_time = time.time() - t0
print(f"\n>>> Full träning och test klar på: {cell_time/60:.1f} minuter")

Dumpa ned modellen för voting.

In [ ]:
import os

# 1. Definiera sökvägen 
model_path = 'C:/mnist_data/mnist_rf_hog_max.joblib'

# 2. Säkerställ att mappen finns
if not os.path.exists('C:/mnist_data'):
    os.makedirs('C:/mnist_data')
    print(">>> Skapade mappen C:/mnist_data")

# 3. Spara modellen
# Vi använder variabeln final_pipe_rf som vi definierade tidigare
joblib.dump(final_pipe_rf, 'C:/mnist_data/mnist_rf_hog_max.joblib', compress=3)

print("-" * 40)
print("Modellen är sparad.")
